# Preprocessing NLP per ticket di supporto

In questo notebook vengono preparati i dati testuali dei ticket per l'addestramento dei modelli di classificazione:
- costruzione del campo testuale (oggetto + descrizione)
- pulizia testuale semplice e riproducibile
- analisi di base delle lunghezze
- suddivisione train/test senza leakage

In [1]:
import pandas as pd
import re

from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"

from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth", 200)

df = pd.read_csv(DATA_DIR / "raw" / "tickets_realistic.csv")

print("Shape iniziale:", df.shape)
df.head()

Shape iniziale: (480, 7)


,id,title,body,category,priority,title_length,body_length
0,1,Mancata notifica: email approvazione persa (processo fermo),bug validazione codice fiscale per unovo cliente Raimondi e figli. Il sys. lo segna come errato anche se è formalmente valido. Impedisce il salvataggio delal scheda cliente. Necsesario bypassare i...,Amministrazione,media,59,248
1,2,Monitoraggio: log modulo utenti (nessun problem),Nota di aggiornamento anagrafica per cliente Micheletti-Fermi s.r.l. nel sistema ordini. Insertii dati nuovo referente operatore_23 a puro titolo informativo per il database. Nessuna azione di sin...,Amministrazione,bassa,48,292
2,3,URGENTE: Account bloccato admin_01 - errore SQL_ERROR_1045 su applicativo reportistica - negoziazione con listino,Monitoraggio passiov dei log modulo gestione utenit per statistiche utilizzo. Analisi a posteriori dei tracciati per identificare trend di lungo periodo. Non ci soon errori attivi o malfunzionamen...,Amministrazione,alta,113,293
3,4,Batch: consolidamento dati contabili per bug con login fallito memory leak,"err. di stampa isolato: impossibile stampare la fat 7243/23. Il sistema genera il PDF correttamente a video, ma l'invio alla stampante fallisce. Altre fatture vengono stampate regolamrente. Il cli...",Amministrazione,media,74,296
4,5,Info: statistiche licenze Linea Pro,Richiesta estrazione dati statistici su utilizzo licenze Linea Pro. Il report serve esclusivamente per proiezioni di budget anno prossimo (uso interno). Nessuna scadenza operativa immediata. Elabo...,Amministrazione,bassa,35,252


## 2. Costruzione del campo testuale unico

Vengono uniti oggetto e descrizione del ticket in un singolo campo `text`, che sarà utilizzato come input testuale per tutti i modelli.


In [2]:
TITLE_COL = "title"      
BODY_COL  = "body"       

# Controllo che le colonne esistano
print("Colonne presenti:", df.columns.tolist())
if TITLE_COL not in df.columns or BODY_COL not in df.columns:
    raise ValueError(f"Le colonne {TITLE_COL} e/o {BODY_COL} non esistono nel dataset. Adattale prima di proseguire.")

# Costruzione del campo testuale
df["text"] = (
    df[TITLE_COL].fillna("").astype(str) + " " + df[BODY_COL].fillna("").astype(str)
).str.strip()

# Rimuovo righe con testo completamente vuoto
before = df.shape[0]
df = df[df["text"].str.len() > 0].copy()
after = df.shape[0]

print(f"Righe totali dopo rimozione testi vuoti: {after} (rimosse {before - after})")
df[["id", TITLE_COL, BODY_COL, "text"]].head()


Colonne presenti: ['id', 'title', 'body', 'category', 'priority', 'title_length', 'body_length']
Righe totali dopo rimozione testi vuoti: 480 (rimosse 0)


,id,title,body,text
0,1,Mancata notifica: email approvazione persa (processo fermo),bug validazione codice fiscale per unovo cliente Raimondi e figli. Il sys. lo segna come errato anche se è formalmente valido. Impedisce il salvataggio delal scheda cliente. Necsesario bypassare i...,Mancata notifica: email approvazione persa (processo fermo) bug validazione codice fiscale per unovo cliente Raimondi e figli. Il sys. lo segna come errato anche se è formalmente valido. Impedisce...
1,2,Monitoraggio: log modulo utenti (nessun problem),Nota di aggiornamento anagrafica per cliente Micheletti-Fermi s.r.l. nel sistema ordini. Insertii dati nuovo referente operatore_23 a puro titolo informativo per il database. Nessuna azione di sin...,Monitoraggio: log modulo utenti (nessun problem) Nota di aggiornamento anagrafica per cliente Micheletti-Fermi s.r.l. nel sistema ordini. Insertii dati nuovo referente operatore_23 a puro titolo i...
2,3,URGENTE: Account bloccato admin_01 - errore SQL_ERROR_1045 su applicativo reportistica - negoziazione con listino,Monitoraggio passiov dei log modulo gestione utenit per statistiche utilizzo. Analisi a posteriori dei tracciati per identificare trend di lungo periodo. Non ci soon errori attivi o malfunzionamen...,URGENTE: Account bloccato admin_01 - errore SQL_ERROR_1045 su applicativo reportistica - negoziazione con listino Monitoraggio passiov dei log modulo gestione utenit per statistiche utilizzo. Anal...
3,4,Batch: consolidamento dati contabili per bug con login fallito memory leak,"err. di stampa isolato: impossibile stampare la fat 7243/23. Il sistema genera il PDF correttamente a video, ma l'invio alla stampante fallisce. Altre fatture vengono stampate regolamrente. Il cli...","Batch: consolidamento dati contabili per bug con login fallito memory leak err. di stampa isolato: impossibile stampare la fat 7243/23. Il sistema genera il PDF correttamente a video, ma l'invio a..."
4,5,Info: statistiche licenze Linea Pro,Richiesta estrazione dati statistici su utilizzo licenze Linea Pro. Il report serve esclusivamente per proiezioni di budget anno prossimo (uso interno). Nessuna scadenza operativa immediata. Elabo...,Info: statistiche licenze Linea Pro Richiesta estrazione dati statistici su utilizzo licenze Linea Pro. Il report serve esclusivamente per proiezioni di budget anno prossimo (uso interno). Nessuna...


## 3. Pulizia testuale semplice

Viene definita una funzione `clean_text` che applica:

- conversione in minuscolo
- rimozione dei simboli non utili
- normalizzazione degli spazi


In [3]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        text = "" if text is None else str(text)
        
    # minuscole (e togli lowercase=False dal TFIDF)
    text = text.lower()
    
    # rimuovi solo simboli non utili
    text = re.sub(r"[^\w\sàèéìòù]", " ", text)
    
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Test veloce della funzione su un esempio
example = "URGENTE: Fattura 12345 non pagata!!!"
print("Originale:", example)
print("Pulito   :", clean_text(example))


Originale: URGENTE: Fattura 12345 non pagata!!!
Pulito   : urgente fattura 12345 non pagata


## 4. Applicazione della pulizia e controlli di base

Applicazione di `clean_text` al campo `text` per ottenere `text_clean` e verifica dei primi 5 record prima/dopo


In [4]:
df["text_clean"] = df["text"].apply(clean_text)

print("Esempi di testo prima/dopo:\n")
for i in range(5):
    print(f"--- Ticket {i} ---")
    print("TEXT      :", df.iloc[i]["text"])
    print("TEXT_CLEAN:", df.iloc[i]["text_clean"])
    print()


Esempi di testo prima/dopo:

--- Ticket 0 ---
TEXT      : Mancata notifica: email approvazione persa (processo fermo) bug validazione codice fiscale per unovo cliente Raimondi e figli. Il sys. lo segna come errato anche se è formalmente valido. Impedisce il salvataggio delal scheda cliente. Necsesario bypassare il controllo o correggere l'algoritmo di validazione.
TEXT_CLEAN: mancata notifica email approvazione persa processo fermo bug validazione codice fiscale per unovo cliente raimondi e figli il sys lo segna come errato anche se è formalmente valido impedisce il salvataggio delal scheda cliente necsesario bypassare il controllo o correggere l algoritmo di validazione

--- Ticket 1 ---
TEXT      : Monitoraggio: log modulo utenti (nessun problem) Nota di aggiornamento anagrafica per cliente Micheletti-Fermi s.r.l. nel sistema ordini. Insertii dati nuovo referente operatore_23 a puro titolo informativo per il database. Nessuna azione di sincronizzazione richiesta al momento. Semplice 

## 5. Analisi delle lunghezze dei testi

Analisi la lunghezza dei testi (in parole) prima e dopo la pulizia al fine di capire quanto contesto avranno a disposizione i modelli


In [5]:
# numero di parole prima e dopo la pulizia
df["len_words_raw"] = df["text"].str.split().str.len()
df["len_words_clean"] = df["text_clean"].str.split().str.len()

summary = df[["len_words_raw", "len_words_clean"]].describe().T
print(summary)

# controllo ticket estremamente corti
print("\nEsempi di ticket molto corti (<= 3 parole pulite):\n")
short_mask = df["len_words_clean"] <= 3
df[short_mask][["id", "text", "text_clean", "len_words_clean"]].head(10)


                 count       mean       std   min   25%   50%   75%   max
len_words_raw    480.0  47.081250  7.126333  30.0  41.0  46.0  52.0  70.0
len_words_clean  480.0  48.191667  7.720370  31.0  43.0  47.0  53.0  79.0

Esempi di ticket molto corti (<= 3 parole pulite):



,id,text,text_clean,len_words_clean


## 6. Suddivisione train/test

Utilizzo di `train_test_split` scikit-learn per dividere il dataset in due parti (training set e test set) sul testo grezzo (`text`) e sulle etichette:

- `category` (classificazione della tipologia di ticket)
- `priority` (bassa / media / alta)

Uso di `stratify` sulla categoria per mantenere una distribuzione simile nel train e nel test.


In [6]:
 # Controllo che le colonne target esistano
for col in ["category", "priority"]:
    if col not in df.columns:
        raise ValueError(f"La colonna target '{col}' non esiste nel dataset. Controlla il generatore di dataset.")

X = df["text"] 

y_cat = df["category"]
y_pri = df["priority"]

X_train, X_test, y_cat_train, y_cat_test = train_test_split(
    X,
    y_cat,
    test_size=0.2,
    random_state=42,
    stratify=y_cat
)

# riallineo anche le priorità usando gli indici
y_pri_train = df.loc[X_train.index, "priority"]
y_pri_test  = df.loc[X_test.index,  "priority"]

print("Dimensioni train/test:")
print("X_train:", X_train.shape[0])
print("X_test :", X_test.shape[0])

Dimensioni train/test:


X_train: 384
X_test : 96


## 7. Controllo della distribuzione delle classi

Verifica della distribuzione categorie affinchè sia simile nel dataset completo, nel train e nel test.


In [7]:
def print_distribution_and_counts(label, series_full, series_train, series_test):
    """Stampa distribuzioni percentuali e conteggi assoluti per full/train/test di una variabile target."""
    
    datasets = {
        "full" : series_full,
        "train": series_train,
        "test" : series_test
    }
    
    print(f"\n=== {label.upper()} ===")

    # Distribuzioni percentuali
    print("\nDistribuzioni percentuali (%):")
    for name, s in datasets.items():
        perc = s.value_counts(normalize=True).mul(100).round(2)
        print(f"\n{name}:")
        print(perc)

    # Conteggi assoluti
    print("\nConteggi assoluti:")
    for name, s in datasets.items():
        print(f"\n{name}:")
        print(s.value_counts())


# Uso della funzione
print_distribution_and_counts("Category", y_cat, y_cat_train, y_cat_test)
print_distribution_and_counts("Priority", y_pri, y_pri_train, y_pri_test)



=== CATEGORY ===

Distribuzioni percentuali (%):

full:
category
Tecnico            45.0
Amministrazione    30.0
Commerciale        25.0
Name: proportion, dtype: float64

train:
category
Tecnico            45.05
Amministrazione    29.95
Commerciale        25.00
Name: proportion, dtype: float64

test:
category
Tecnico            44.79
Amministrazione    30.21
Commerciale        25.00
Name: proportion, dtype: float64

Conteggi assoluti:

full:
category
Tecnico            216
Amministrazione    144
Commerciale        120
Name: count, dtype: int64

train:
category
Tecnico            173
Amministrazione    115
Commerciale         96
Name: count, dtype: int64

test:
category
Tecnico            43
Amministrazione    29
Commerciale        24
Name: count, dtype: int64

=== PRIORITY ===

Distribuzioni percentuali (%):

full:
priority
media    38.12
alta     32.92
bassa    28.96
Name: proportion, dtype: float64

train:
priority
media    40.62
alta     33.85
bassa    25.52
Name: proportion, dtype

## 8. Salvataggio del dataset con indicazione train/test

Viene aggiunta una colonna `split` al DataFrame (`train` / `test`) e salvato un CSV aggiornato, che sarà il punto di partenza per EDA e training.


In [8]:
# inizialmente tutto train
df["split"] = "train"

# assegno 'test' alle righe presenti in X_test
df.loc[X_test.index, "split"] = "test"

print(df["split"].value_counts())

# TODO: adatta il path/output
output_path = DATA_DIR / "splits" / "tickets_preprocessed_split.csv"
df.to_csv(output_path, index=False)
print(f"\nFile salvato in: {output_path}")


split
train    384
test      96
Name: count, dtype: int64

File salvato in: C:\project-work\data\splits\tickets_preprocessed_split.csv
